<a href="https://colab.research.google.com/github/henrykohl/bentoml-demo/blob/main/sklearn-sentiment-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BentoML Example: Sentiment Analysis with Scikit-learn

* 根據 -- 參2

* [Dataset](https://docs.google.com/file/d/0B04GJPshIjmPRnZManQwWEdTZjg/edit?resourcekey=0-betyQkEmWZgp8z0DFxWsHw) Already save a copy file in my google drive (under the `FileDisk` folder)

* Requried packages with theirversions:

> python=3.7
>
> bentoml=0.9.0
>
> protobuf=3.20.*
>
> sqlalchemy=1.3.*

## 安裝Conda

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


## 取得 trainingandtestdata.zip

In [ ]:
import os

from google.colab import drive

drive.mount('/content/drive/')
os.chdir('/content/drive/MyDrive/FileDisk')
# os.listdir()
# !mv trainingandtestdata.zip /content
!cp trainingandtestdata.zip /content/trainingandtestdata.zip
os.chdir('/content') ## 切換到 colab 使用者 default 目錄

Mounted at /content/drive/


In [ ]:
# 似乎沒用到
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## 解壓 trainingandtestdata.zip

In [ ]:
%%bash
unzip -n trainingandtestdata.zip

Archive:  trainingandtestdata.zip
  inflating: testdata.manual.2009.06.14.csv  
  inflating: training.1600000.processed.noemoticon.csv  


## 用 Conda 建立 virtual environment

In [ ]:
!conda create -p env python=3.7 -y -q

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /content/env

  added / updated specs:
    - python=3.7


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.1.31  |       hbd8a1cb_1         155 KB  conda-forge
    ld_impl_linux-64-2.43      |       h712a8e2_4         656 KB  conda-forge
    libffi-3.4.6               |       h2dba641_1          56 KB  conda-forge
    libgcc-14.2.0              |       h767d61c_2         828 KB  conda-forge
    libgcc-ng-14.2.0           |       h69a702a_2          52 KB  conda-forge
    libgomp-14.2.0             |       h767d61c_2         449 KB  conda-forge
    liblzma-5.8.1              |       hb9d3cd8_0         110 KB  conda-forge
    liblzma-devel-5.8.1        |       hb9d3cd8_0         431 KB  conda-forge
    libsqlite-3.49.1           |       hee588

## 在 env 下，安裝 `bentoml(0.9.0)`

In [ ]:
!source activate ./env; pip install -q 'bentoml==0.9.0'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 988.0/988.0 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.8/99.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 17.9 MB/s eta 0:00:00
   ━━━

## 在 env 下，安裝 `scipy` 與 `matplotlib`

In [ ]:
!source activate ./env; pip install -U scipy matplotlib --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.3 MB/s eta 0:00:00


## 在 env 下，安裝 `scikit-learn` 與 `pandas` 與 `numpy`

In [ ]:
!source activate ./env; pip install -q 'scikit-learn>=0.23.2' 'pandas>=1.1.1' 'numpy>=1.8.2'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 25.6 MB/s eta 0:00:00


## 在 env 下，安裝 `protoful(3.20.X)`

In [ ]:
!source activate ./env; pip install protobuf==3.20.* -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.4 MB/s eta 0:00:00


## 在 env 下，安裝 `sqlalchemy(1.3.*)`

In [ ]:
!source activate ./env; pip install -q sqlalchemy==1.3.*

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.7 MB/s eta 0:00:00


## 訓練模型 sentiment_lr

用 `pickle` 將模型 sentiment_lr 存起來成為 `sentiment_lf.pkl`

* 重要參考(註1)

In [ ]:
%%bash
source activate ./env

python

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline

import bentoml

log_model = LogisticRegression(solver='lbfgs', max_iter=1000)

columns = ['polarity', 'tweetid', 'date', 'query_name', 'user', 'text']
dftrain = pd.read_csv('training.1600000.processed.noemoticon.csv',
                      header = None,
                      encoding ='ISO-8859-1')
dftest = pd.read_csv('testdata.manual.2009.06.14.csv',
                     header = None,
                     encoding ='ISO-8859-1')
dftrain.columns = columns
dftest.columns = columns

sentiment_lr = Pipeline([
                         ('count_vect', CountVectorizer(min_df = 100,
                                                        ngram_range = (1,2),
                                                        stop_words = 'english')),
                         ('lr', log_model)])
sentiment_lr.fit(dftrain.text, dftrain.polarity)

Xtest, ytest = dftest.text[dftest.polarity!=2], dftest.polarity[dftest.polarity!=2]
print(classification_report(ytest,sentiment_lr.predict(Xtest)))

# sentiment_lr.predict([Xtest[0]])
result=sentiment_lr.predict([Xtest[0]])
print("TYPE:", type(sentiment_lr))
print("Input data=",Xtest[0],", result=",result)

"""以下是自行新增的部分(註**)"""
import pickle

# save the Pipeline as a pickle file
pipeline_pkl_file = "sentiment_lf.pkl"

with open(pipeline_pkl_file, 'wb') as file:
    pickle.dump(sentiment_lr, file)

              precision    recall  f1-score   support

           0       0.86      0.81      0.83       177
           4       0.82      0.87      0.85       182

    accuracy                           0.84       359
   macro avg       0.84      0.84      0.84       359
weighted avg       0.84      0.84      0.84       359

TYPE: <class 'sklearn.pipeline.Pipeline'>
Input data= @stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right. , result= [4]


<font color="red">(註**) 的作用是將 `sentiment_lr` 存到檔案中，而當下一個 cell 再次執行 `source activate ./env` 時，模型 `sentiment_lr` 已經不存在，還是可以利用讀取檔案 "sentiment_lf.pkl" ，來恢復模型 `sentiment_lr`</font>

## 建立 BentoService API

In [ ]:
%%writefile sentiment_analysis_service.py
import pandas as pd
import bentoml
from bentoml.frameworks.sklearn import SklearnModelArtifact
from bentoml.service.artifacts.common import PickleArtifact
from bentoml.handlers import DataframeHandler
from bentoml.adapters import DataframeInput

@bentoml.artifacts([PickleArtifact('model')])
@bentoml.env(pip_packages=["scikit-learn", "pandas"])
class SKSentimentAnalysis(bentoml.BentoService):

    @bentoml.api(input=DataframeInput(), batch=True)
    def predict(self, df):
        """
        predict expects pandas.Series as input
        """
        series = df.iloc[0,:]
        return self.artifacts.model.predict(series)

Overwriting sentiment_analysis_service.py


## 保存 BentoService 到檔案中

BentoService 把 sentiment_lf 模型打包

In [ ]:
%%bash
source activate ./env

python

from sentiment_analysis_service import SKSentimentAnalysis
import pickle

pipeline_pkl_file = "sentiment_lf.pkl"

with open(pipeline_pkl_file, 'rb') as file:
    sentiment_lf = pickle.load(file)

bento_service = SKSentimentAnalysis()
bento_service.pack('model', sentiment_lf)

saved_path = bento_service.save() ## Finally successful

# train 過程中產生的 "saved_path"，被存到'var_obj.pkl'中
var_file = "var_obj.pkl"

with open(var_file, 'wb') as file:
    pickle.dump(saved_path, file)

[2025-04-22 06:35:56,067] WARNING - bentoml.handlers.* will be deprecated after BentoML 1.0, use bentoml.adapters.* instead
[2025-04-22 06:36:01,390] WARNING - pip package requirement pandas already exist
[2025-04-22 06:36:29,304] INFO - BentoService bundle 'SKSentimentAnalysis:20250422063602_F5F7CA' saved to: /root/bentoml/repository/SKSentimentAnalysis/20250422063602_F5F7CA


In [ ]:
!source activate ./env; bentoml list

BENTO_SERVICE                              AGE            APIS                                   ARTIFACTS              LABELS
SKSentimentAnalysis:20250422063602_F5F7CA  14.76 seconds  predict<DataframeInput:DefaultOutput>  model<PickleArtifact>


## 啟動 REST API model server 用上一節保存的 BentoService

In [ ]:
!source activate ./env; bentoml serve SKSentimentAnalysis:latest

[2025-03-03 08:17:13,426] INFO - Getting latest version SKSentimentAnalysis:20250303072553_3E9744
[2025-03-03 08:17:13,427] INFO - Starting BentoML API server in development mode..
[2025-03-03 08:17:14,404] WARNING - bentoml.handlers.* will be deprecated after BentoML 1.0, use bentoml.adapters.* instead
[2025-03-03 08:17:19,282] WARNING - pip package requirement pandas already exist
 * Serving Flask app 'SKSentimentAnalysis'
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit

Aborted!


### 另法，但需要註冊 ngrok

In [ ]:
!source activate ./env; bentoml serve SKSentimentAnalysis:latest --run-with-ngrok

## Load saved BentoService

In [ ]:
%%bash

source activate ./env

python

import bentoml
import pandas as pd

# saved_path = "/root/bentoml/repository/SKSentimentAnalysis/20250303095521_E343F0" ## 測試用

# train 過程中產生的 "saved_path"，被存到'var_obj.pkl'中，在此再從'var_obj.pkl'存取 "saved_path"
import pickle
var_file = "var_obj.pkl"

with open(var_file, 'rb') as file:
    saved_path = pickle.load(file)

# Load exported bentoML model archive from path
loaded_bento_service = bentoml.load(saved_path)

# Call predict on the restored sklearn model
result=loaded_bento_service.predict(pd.DataFrame(data=["good", "great"]))
print("result=",result)

[2025-03-03 10:56:01,951] WARNING - bentoml.handlers.* will be deprecated after BentoML 1.0, use bentoml.adapters.* instead
[2025-03-03 10:56:06,151] WARNING - pip package requirement pandas already exist
result= [4]


## Launch inference job from CLI

In [ ]:
!source activate ./env; bentoml run SKSentimentAnalysis:latest predict \
--input '["some new text, sweet noodles", "happy time", "sad day"]'

[2025-03-03 08:45:36,070] INFO - Getting latest version SKSentimentAnalysis:20250303072553_3E9744
[2025-03-03 08:45:36,723] WARNING - bentoml.handlers.* will be deprecated after BentoML 1.0, use bentoml.adapters.* instead
[2025-03-03 08:45:41,846] WARNING - pip package requirement pandas already exist
[2025-03-03 08:45:42,363] INFO - {'service_name': 'SKSentimentAnalysis', 'service_version': '20250303072553_3E9744', 'api': 'predict', 'task': {'data': '["some new text, sweet noodles", "happy time", "sad day"]', 'task_id': 'd2b2302b-1372-4ca2-9b82-d492d3c80413', 'batch': 3, 'cli_args': ('--input', '["some new text, sweet noodles", "happy time", "sad day"]')}, 'result': {'data': '[4, 4, 4]', 'http_status': 200, 'http_headers': (('Content-Type', 'application/json'),)}, 'request_id': 'd2b2302b-1372-4ca2-9b82-d492d3c80413'}
[4, 4, 4]


# BentoML Example 02: Iris Prediction



* 根據 -- 參3

* Requried packages with theirversions:

> python=3.10.11
>
> bentoml=1.0.19
>
> scikit-learn==1.2.2
>
> numpy==1.26.4



In [17]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
!conda create -p env python=3.10.11 -y -q

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /content/env

  added / updated specs:
    - python=3.10.11


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.1.31  |       hbcca054_0         154 KB  conda-forge
    ld_impl_linux-64-2.43      |       h712a8e2_4         656 KB  conda-forge
    libffi-3.4.6               |       h2dba641_1          56 KB  conda-forge
    libgcc-14.2.0              |       h767d61c_2         828 KB  conda-forge
    libgcc-ng-14.2.0           |       h69a702a_2          52 KB  conda-forge
    libgomp-14.2.0             |       h767d61c_2         449 KB  conda-forge
    liblzma-5.8.1              |       hb9d3cd8_0         110 KB  conda-forge
    liblzma-devel-5.8.1        |       hb9d3cd8_0         431 KB  conda-forge
    libsqlite-3.49.1           |       he

In [ ]:
!source activate ./env; pip install -q 'bentoml==1.0.19'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.2/981.2 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.5/862.5 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.0 MB/s eta 0:00:00


In [ ]:
!source activate ./env; pip install -q scikit-learn==1.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 35.1 MB/s eta 0:00:00


In [ ]:
!source activate ./env; pip install -U pandas --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 37.5 MB/s eta 0:00:00


In [ ]:
!source activate ./env; pip install -q numpy==1.26.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 87.9 MB/s eta 0:00:00


---

In [ ]:
%%bash
source activate ./env

python

from sklearn import svm, datasets

# Load training data
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Model Training
clf = svm.SVC()
clf.fit(X, y)


import pickle

# save the iris classification model as a pickle file
model_pkl_file = "clf.pkl"

with open(model_pkl_file, 'wb') as file:
    pickle.dump(clf, file)

In [ ]:
%%bash
source activate ./env

python

import pickle

model_pkl_file = "clf.pkl"

with open(model_pkl_file, 'rb') as file:
    clf = pickle.load(file)


import bentoml

saved_model = bentoml.sklearn.save_model("iris_clf", clf);

In [ ]:
!source activate ./env; bentoml models list

 Tag                        Module           Size      Creation Time       
 iris_clf:724q4lq3uovxuasc  bentoml.sklearn  6.03 KiB  2025-04-17 15:52:41 


In [ ]:
%%bash
source activate ./env

python

import bentoml

loaded_model = bentoml.sklearn.load_model("iris_clf:latest")

result=loaded_model.predict([[5.9, 3.0, 5.1, 1.8]])
print("result:",result)

result: [2]


In [ ]:
%%bash
source activate ./env

python

import bentoml

# 创建Runner实例：
iris_clf_runner = bentoml.sklearn.get("iris_clf:latest").to_runner()

# “Runner.init_local”仅用于调试和测试。确保在部署到生产环境之前将其移除。
iris_clf_runner.init_local()

# 这将产生与加载模型相同的结果：
result = iris_clf_runner.predict.run([[5.9, 3.0, 5.1, 1.8]])
print("result:", result)

result: [2]


'Runner.init_local' is for debugging and testing only. Make sure to remove it before deploying to production.


In [ ]:
%%writefile service.py
import numpy as np
import bentoml
from bentoml.io import NumpyNdarray

# Runner实例化
iris_clf_runner = bentoml.sklearn.get("iris_clf:latest").to_runner()

# 定义服务名称并加载Runner实例
svc = bentoml.Service("iris_classifier", runners=[iris_clf_runner])

# 与FastAPI一致，通过装饰器绑定路由与视图
@svc.api(input=NumpyNdarray(), output=NumpyNdarray())
def classify(input_series: np.ndarray) -> np.ndarray:
    return iris_clf_runner.predict.run(input_series)

Writing service.py


In [ ]:
%%writefile bentofile.yaml
service: "service.py:svc"
labels:
    owner: bentoml-team
    project: gallery
include:
- "*.py"
python:
    packages:
        - scikit-learn
        - pandas

Writing bentofile.yaml


* TIP: **include** 不用 `"*.py"` ，明確寫出哪些 py 檔，在 `bentoml build` 時，會比較快/比較不會出問題

In [ ]:
!source activate ./env; bentoml models get iris_clf:latest

name: iris_clf
version: 724q4lq3uovxuasc
module: bentoml.sklearn
labels: {}
options: {}
metadata: {}
context:
  framework_name: sklearn
  framework_versions:
    scikit-learn: 1.2.2
  bentoml_version: 1.0.19
  python_version: 3.10.11
signatures:
  predict:
    batchable: false
api_version: v1
creation_time: '2025-04-17T15:52:41.565225+00:00'



In [ ]:
!source activate ./env; bentoml serve service.py:svc --reload

2025-04-17T16:49:59+0000 [INFO] [cli] Prometheus metrics for HTTP BentoServer from "service.py:svc" can be accessed at http://localhost:3000/metrics.
2025-04-17T16:49:59+0000 [INFO] [cli] Starting development HTTP BentoServer from "service.py:svc" listening on http://0.0.0.0:3000 (Press CTRL+C to quit)
2025-04-17 16:50:00 circus[16123] [INFO] Loading the plugin...
2025-04-17 16:50:00 circus[16123] [INFO] Endpoint: 'tcp://127.0.0.1:43143'
2025-04-17 16:50:00 circus[16123] [INFO] Pub/sub: 'tcp://127.0.0.1:34141'
2025-04-17T16:50:00+0000 [INFO] [observer] Watching directories: ['/content', '/root/bentoml/models']


In [ ]:
!source activate ./env; bentoml build

Building BentoML service "iris_classifier:x6xqxkq3v6vlqasc" from build context "/content".
Packing model "iris_clf:724q4lq3uovxuasc"

Aborted!


In [ ]:
!source activate ./env; bentoml containerize iris_classifier:latest

---

# BentoML Example 02-1: Iris Prediction

* 根據 -- 參4

python 3.8 或 3.9 都可以，但 bentoml 要 1.3.0 (1.0.15/1.0.19/1.0.25/1.2.0都會出錯)，1.3.10會出現警告

* python 3.8 + bentoml==1.3.0: default 配 numpy 1.24.4

* python 3.9 + bentoml==1.3.0: default 配 numpy 2.0.2

In [25]:
!conda create -p venv python=3.9 -y -q
# !conda create -p venv python=3.8 -y -q

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /content/venv

  added / updated specs:
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    liblzma-devel-5.8.1        |       hb9d3cd8_0         431 KB  conda-forge
    pip-24.3.1                 |     pyh8b19718_0         1.2 MB  conda-forge
    python-3.8.20              |h4a871b0_2_cpython        21.1 MB  conda-forge
    setuptools-75.3.0          |     pyhd8ed1ab_0         761 KB  conda-forge
    wheel-0.45.1               |     pyhd8ed1ab_0          62 KB  conda-forge
    xz-5.8.1                   |       hbcc6ac9_0          23 KB  conda-forge
    xz-gpl-tools-5.8.1         |       hbcc6ac9_0          33 KB  conda-forge
    xz-tools-5.8.1             |       hb9d3cd8_0          94 KB  conda-forge
    ---------------------------------------

In [27]:
# !source activate ./venv; pip install -q bentoml==1.0.25 # bentoml list有KeyError(python 3.8/3.9 )
# !source activate ./venv; pip install -q bentoml==1.3.10 # 有一些警告
!source activate ./venv; pip install -q bentoml==1.3.0 # 好像沒有警告
# ----------------------------------------------------------------------
# !source activate ./venv; pip install -q bentoml==1.2.0 # 不行
# !source activate ./venv; pip install -q bentoml==1.0.15 # 不行

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.5/746.5 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 140.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.4/863.4 kB 30.3 MB/s eta 0:00:00


In [11]:
# !source activate ./venv; pip install -q scikit-learn==1.2.2 #(python 3.8)
!source activate ./venv; pip install -q scikit-learn==1.6.1 pandas==2.2.3 #(python 3.9)
# !source activate ./venv; pip install -q scikit-learn==0.23.2 #(python 3.8)

In [8]:
!source activate ./venv; pip install -q pandas==1.1.1 #(python 3.8)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 68.4 MB/s eta 0:00:00


In [7]:
# python 3.8 或 3.9 時，此步驟是不需要的
# !source activate ./venv; pip install -q numpy==1.26.4
!source activate ./venv; pip install -q numpy==1.20.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 84.2 MB/s eta 0:00:00


In [13]:
!source activate ./venv; pip install -q pydantic==2.5.1 # (Lecture 缺) 必需的

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.3 MB/s eta 0:00:00


In [6]:
%%writefile train.py
from sklearn import svm
from sklearn import datasets
import bentoml

# Load training data
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Model Training
clf = svm.SVC(gamma='scale')
clf.fit(X, y)

# 调用`bentoml.<FRAMEWORK>.save(<MODEL_NAME>, model)` 为了在本地模型存储中保存为 BentoML 的标准格式
# bentoml.sklearn.save("iris_clf", clf) # deprecated
bentoml.sklearn.save_model("iris_clf", clf)

Writing train.py


In [14]:
!source activate ./venv; python train.py

In [15]:
!source activate ./venv; bentoml models list

 Tag                        Module           Size      Creation Time       
 iris_clf:yyhuhjba2wff4asc  bentoml.sklearn  5.99 KiB  2025-04-24 06:31:37 


In [16]:
%%writefile verify.py

import bentoml

runner = bentoml.sklearn.get("iris_clf:latest").to_runner()

runner.init_local()

print(runner.predict.run([[5.9, 3., 5.1, 1.8]]))  # => array(2)

Writing verify.py


In [17]:
!source activate ./venv; python verify.py

'Runner.init_local' is for debugging and testing only. Make sure to remove it before deploying to production.
[2]


In [18]:
%%writefile service.py
import numpy as np
import bentoml
from bentoml.io import NumpyNdarray

# 加载我们刚刚保存的最新 ScikitLearn 模型的运行器
iris_clf_runner = bentoml.sklearn.get("iris_clf:latest").to_runner()

# 使用 ScikitLearn 运行器创建 iris_classifier 服务
# 如果需要，可以在runners数组中指定多个运行器
# 当包装为bento时，运行器（runners）也会被包括在里面
svc = bentoml.Service("iris_classifier", runners=[iris_clf_runner])


# 使用“svc”注解创建具有预处理和后处理逻辑的 API 函数
@svc.api(input=NumpyNdarray(), output=NumpyNdarray())
def classify(input_series: np.ndarray) -> np.ndarray:
    # 定义预处理逻辑
    result = iris_clf_runner.predict.run(input_series)
    # 定义后处理逻辑
    return result

Writing service.py


In [19]:
!source activate ./venv; bentoml serve service.py:svc --reload

2025-04-24T06:32:01+0000 [INFO] [cli] Environ for worker 0: set CPU thread count to 2
2025-04-24T06:32:01+0000 [INFO] [cli] Prometheus metrics for HTTP BentoServer from "service.py:svc" can be accessed at http://localhost:3000/metrics.
2025-04-24T06:32:01+0000 [INFO] [cli] Starting production HTTP BentoServer from "service.py:svc" listening on http://0.0.0.0:3000 (Press CTRL+C to quit)


In [34]:
!source activate ./venv; curl \
  -X POST \
  -H "content-type: application/json" \
  --data "[5,4,3,2]" \
  http://127.0.0.1:3000/classify

curl: /usr/local/lib/libcurl.so.4: no version information available (required by curl)
curl: (7) Failed to connect to 127.0.0.1 port 3000 after 0 ms: Could not connect to server


In [20]:
%%writefile bentofile.yaml
service: "service.py:svc"  # 定位服务的约定：<YOUR_SERVICE_PY>:<YOUR_SERVICE_ANNOTATION>
labels:
    owner: bentoml-team
    stage: demo
include:
  - "service.py"
python:
    packages:
    - scikit-learn
    - pandas
# models:              # 可有可無(如果 store 中只有一個 model)
# - iris_clf:latest

Writing bentofile.yaml


In [21]:
!source activate ./venv; bentoml build

INFO: Adding current BentoML version to requirements.txt: 1.3.0
INFO: Locking PyPI package versions.

██████╗ ███████╗███╗   ██╗████████╗ ██████╗ ███╗   ███╗██╗
██╔══██╗██╔════╝████╗  ██║╚══██╔══╝██╔═══██╗████╗ ████║██║
██████╔╝█████╗  ██╔██╗ ██║   ██║   ██║   ██║██╔████╔██║██║
██╔══██╗██╔══╝  ██║╚██╗██║   ██║   ██║   ██║██║╚██╔╝██║██║
██████╔╝███████╗██║ ╚████║   ██║   ╚██████╔╝██║ ╚═╝ ██║███████╗
╚═════╝ ╚══════╝╚═╝  ╚═══╝   ╚═╝    ╚═════╝ ╚═╝     ╚═╝╚══════╝

Successfully built Bento(tag="iris_classifier:4itifdba2wwdeasc").

Next steps: 

* Deploy to BentoCloud:
    $ bentoml deploy iris_classifier:4itifdba2wwdeasc -n ${DEPLOYMENT_NAME}

* Update an existing deployment on BentoCloud:
    $ bentoml deployment update --bento iris_classifier:4itifdba2wwdeasc 
${DEPLOYMENT_NAME}

* Containerize your Bento with `bentoml containerize`:
    $ bentoml containerize iris_classifier:4itifdba2wwdeasc 

* Push to BentoCloud with `bentoml push`:
    $ bentoml push iris_classifier:4itifdba2wwdeasc

In [22]:
!source activate ./venv; bentoml list

 Tag                               Size       Model Size  Creation Time       
 iris_classifier:4itifdba2wwdeasc  17.03 KiB  5.99 KiB    2025-04-24 06:32:26 


* 若執行 `bentoml list` 時， 出現 KeyError的錯誤，暫時解沒有特殊方式解決。只能安裝互相匹配的 python 版本與 必要的 packages 版本

# BentoML SageMaker deployment

* 根據 -- 參5

In [3]:
!conda create -p venv python==3.12.9 -y -q

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /content/venv

  added / updated specs:
    - python==3.12.9


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.1.31  |       hbd8a1cb_1         155 KB  conda-forge
    ld_impl_linux-64-2.43      |       h712a8e2_4         656 KB  conda-forge
    libexpat-2.7.0             |       h5888daf_0          73 KB  conda-forge
    libffi-3.4.6               |       h2dba641_1          56 KB  conda-forge
    libgcc-14.2.0              |       h767d61c_2         828 KB  conda-forge
    libgcc-ng-14.2.0           |       h69a702a_2          52 KB  conda-forge
    libgomp-14.2.0             |       h767d61c_2         449 KB  conda-forge
    liblzma-5.8.1              |       hb9d3cd8_0         110 KB  conda-forge
    libsqlite-3.49.1           |       h

In [4]:
%%writefile requirements.txt
bentoctl==0.4.0
bentoml==1.4.10
boto3==1.29.0
numpy==1.26.2
pydantic==2.5.1
pydantic_core==2.14.3
scikit-learn==1.3.2

Writing requirements.txt


In [5]:
!source activate ./venv; pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 129.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.5/767.5 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 855.9/855.9 kB 42.3 MB/s eta 0:00:00


In [6]:
!source activate ./venv; pip show cattrs # (檢視) Version: 23.1.2

Name: cattrs
Version: 23.1.2
Summary: Composable complex class support for attrs and dataclasses.
Home-page: https://github.com/python-attrs/cattrs
Author: Tin Tvrtkovic
Author-email: tinchester@gmail.com
License: MIT
Location: /content/venv/lib/python3.12/site-packages
Requires: attrs
Required-by: bentoml


In [ ]:
%%writefile create_model.py
import bentoml

from sklearn import datasets
from sklearn import svm

iris = datasets.load_iris()
X, y = iris.data, iris.target

clf = svm.SVC(gamma="scale")
clf.fit(X, y)

saved_model = bentoml.sklearn.save_model("iris_clf", clf)
print(saved_model)

In [7]:
!source activate ./venv; python create_model.py

Model(tag="iris_clf:mnabqvralgwzmasc")


In [8]:
!source activate ./venv; bentoml models list

 Tag                        Module           Size      Creation Time       
 iris_clf:mnabqvralgwzmasc  bentoml.sklearn  5.98 KiB  2025-04-23 15:41:13 


In [9]:
%%writefile service.py
from typing import Literal

import bentoml

from pydantic import BaseModel
from bentoml.io import JSON


iris_clf_runner = bentoml.sklearn.get("iris_clf:latest").to_runner()

svc = bentoml.Service("iris_classifier", runners=[iris_clf_runner])

class Request(BaseModel):
    sepal_width: float
    sepal_length: float
    petal_width: float
    petal_length: float

class Response(BaseModel):
    label: Literal["setosa", "versicolor", "virginica"]


@svc.api(input=JSON(pydantic_model=Request), output=JSON(pydantic_model=Response))
def classify(request: Request) -> Response:
    input_ = [
        request.sepal_width,
        request.sepal_length,
        request.petal_width,
        request.petal_length,
    ]

    label_index = iris_clf_runner.predict.run([input_])[0]
    label = ["setosa", "versicolor", "virginica"][label_index]

    return Response(label=label)

Writing service.py


In [10]:
%%writefile bentofile.yaml
service: "service:svc"
include:
- "service.py"
python:
  packages:
  - pydantic
  - scikit-learn
models:
- iris_clf:latest

Writing bentofile.yaml


In [11]:
!source activate ./venv; bentoml build

/content/venv/lib/python3.12/site-packages/bentoml/io.py:7: BentoMLDeprecationWarning: `bentoml.io` is deprecated since BentoML v1.4 and will be removed in a future version. Please upgrade to new style IO types instead.
  warn_deprecated(
/content/venv/lib/python3.12/site-packages/bentoml/_internal/models/model.py:367: BentoMLDeprecationWarning: `get_runnable` is deprecated since BentoML v1.4 and will be removed in a future version. Use `get_service` instead.
  self._runnable = self.info.imported_module.get_runnable(self)
/content/venv/lib/python3.12/site-packages/bentoml/_internal/models/model.py:354: BentoMLDeprecationWarning: `Runner` is deprecated since BentoML v1.4 and will be removed in a future version. Please upgrade to new style services.
  return Runner(
/content/service.py:11: BentoMLDeprecationWarning: `bentoml.Service` is deprecated since BentoML v1.4 and will be removed in a future version. Please upgrade to @bentoml.service().
  svc = bentoml.Service("iris_classifier", r

In [12]:
!source activate ./venv; bentoml list

 Tag                               Size       Model Size  Creation Time       
 iris_classifier:6tvcmpralghckasc  19.07 KiB  5.98 KiB    2025-04-23 15:45:20 


# 參考

* 註 1 : [Sklearn中Pipeline的用法介绍](https://blog.csdn.net/WHYbeHERE/article/details/125074001)

* 參 1 : [BentoML Explained: An Optimal Model Serving Framework](https://www.axelmendoza.com/posts/bentoml-core-concepts-and-features/)

* 參 2 : [BentoML Example: Sentiment Analysis with Scikit-learn (Notebook on Google Colab)](https://colab.research.google.com/github/bentoml/gallery/blob/0.13-LTS/scikit-learn/sentiment-analysis/sklearn-sentiment-analysis.ipynb)

* 參 3 : [BentoML模型部署--稀土掘金](https://juejin.cn/post/7231432555320016955)

* 參 4 : [快速入门统一模型部署（Model Serving）框架 BentoML](https://zhuanlan.zhihu.com/p/495814838)～與參三基本相同，但有較多註釋。

* 參 5: [BentoML SageMaker deployment -- Youtube](https://www.youtube.com/watch?v=Zci_D4az9FU)

